In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
from typing import Dict, List
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import f1_score, accuracy_score, classification_report
from torch import nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import torch.nn.functional as F
import sklearn
import sys
import random
import math
import os
import argparse
import logging
from sklearn.utils import class_weight
from torch.cuda.amp import autocast, GradScaler
from transformers import BertModel
from torch.cuda.amp import autocast, GradScaler
from transformers import get_linear_schedule_with_warmup, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

def seed_everything(seed_value=42):
    os.environ["PYTHONHASHSEED"] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    return seed_value


class DataPreparation(Dataset):
    
    def __init__(self, tokenizer, data, intelligence='verb', max_length=None):
        
        self.tokenizer = tokenizer
        self.data = data
        self.intell = intelligence
        
        if max_length == None:
            max_length_counted = data["text"].str.split(' ').str.len().max(axis=0)
            self.max_length = max_length_counted if max_length_counted < 512 else 512
        else:
            self.max_length = max_length


    def __len__(self):
        return len(self.data)


    def tokenize(self, text):

        tokens = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt')

        return tokens

     
    def __getitem__(self, index):
        
        source_text = self.data['text'].iloc[index]
        source = self.tokenize(source_text)

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()

        label = self.data[self.intell].iloc[index]
        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "labels":  label, 
        }


class RNN_Block(nn.Module):
    
    def __init__(self, input_size=768, hidden_size=512, rnn='LSTM', biderectional=True):
        super().__init__()

        rnn_type = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}
        self.rnn = rnn_type[rnn](input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=biderectional)


    def forward(self, input_ids):
        
        rnn_output = self.rnn(input_ids)
        return rnn_output


class CNN_Block(nn.Module):
    
    def __init__(self, input_size=1024, out_size=64, kernel_size=3, stride=1, padding=1):
        super().__init__()

        self.conv_2 = nn.Conv1d(in_channels=input_size, out_channels=out_size,
                                kernel_size=2, stride=stride, padding=padding)
        self.conv_3 = nn.Conv1d(in_channels=input_size, out_channels=out_size,
                                kernel_size=3, stride=stride, padding=padding)
        self.conv_5 = nn.Conv1d(in_channels=input_size, out_channels=out_size,
                                kernel_size=5, stride=stride, padding=padding+1)
        self.relu = nn.ReLU()


    def forward(self, sequence_input):

        conv_input = sequence_input.permute(0, 2, 1) # batch_size, hidden_size, sequence_length
        cnn_output2 = self.conv_2(conv_input)
        cnn_output2 = self.relu(cnn_output2)
        cnn_output2 = F.max_pool1d(cnn_output2, kernel_size=cnn_output2.shape[2])
        cnn_output3 = self.conv_3(conv_input)
        cnn_output3 = self.relu(cnn_output3)
        cnn_output3 = F.max_pool1d(cnn_output3, kernel_size=cnn_output3.shape[2])
        cnn_output5 = self.conv_5(conv_input)
        cnn_output5 = self.relu(cnn_output5)
        cnn_output5 = F.max_pool1d(cnn_output5, kernel_size=cnn_output5.shape[2])
        cnn_output = torch.cat([cnn_output2.squeeze(dim=2), cnn_output3.squeeze(dim=2), cnn_output5.squeeze(dim=2)], dim=1)

        return cnn_output


class BIGRU_BILSTM_CNN(nn.Module):
    
    def __init__(self, out_features, hidden_size=512, hidden_size_lin=128,
                 p_spatial_dropout=0.5, out_chanels_cnn=64, kernel_size_cnn=3, 
                 stride_cnn=1, padding_cnn=1, 
                 rnn_type='LSTM', biderectional=True, pre_trained='DeepPavlov/rubert-base-cased-sentence'):
        super(BIGRU_BILSTM_CNN, self).__init__()

        self.bert = AutoModel.from_pretrained(pre_trained)
        self.layer_norm = nn.LayerNorm(32)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.p_spatial_dropout = p_spatial_dropout
        self.bigru_block = RNN_Block(input_size=self.bert.config.hidden_size, hidden_size=hidden_size, rnn='GRU', biderectional=True) 
        self.bilstm_block = RNN_Block(input_size=self.bert.config.hidden_size, hidden_size=hidden_size, rnn='LSTM', biderectional=True) 
        
        if biderectional == True:
            self.cnn_block = CNN_Block(input_size=hidden_size*2, out_size=out_chanels_cnn,
                                      kernel_size=kernel_size_cnn, stride=stride_cnn, padding=padding_cnn)
        else:
            self.cnn_block = CNN_Block(input_size=hidden_size, out_size=out_chanels_cnn,
                                      kernel_size=kernel_size_cnn, stride=stride_cnn, padding=padding_cnn)

        self.linear_modules = nn.ModuleList([torch.nn.Linear(out_chanels_cnn*3*2, 32),
                                          torch.nn.Linear(32, 32),
                                          torch.nn.Linear(32, out_features)])
                                          
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def forward(self, input_ids, attention_mask):


        encoded_layers = self.bert(input_ids=input_ids,attention_mask=attention_mask, output_hidden_states=True)
        x = encoded_layers['last_hidden_state']#.permute(1, 0, 2)
#         # spatial dropout

        embeddings = x.unsqueeze(2)    # (N, T, 1, K)
        embeddings = embeddings.permute(0, 3, 2, 1)  # (N, K, 1, T)
        embeddings = F.dropout2d(embeddings, self.p_spatial_dropout)  # (N, K, 1, T), some features are masked
        embeddings = embeddings.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = embeddings.squeeze(2)  # (N, T, K)
        bigru,_ = self.bigru_block(x) #(N,L,H in) - batch size, sequence length, input size
        
        bilstm, hidden_bilstm = self.bilstm_block(x)
        bilstm_cnn = self.cnn_block(bilstm)
        bigru_cnn = self.cnn_block(bigru)
        x = torch.cat([bigru_cnn, bilstm_cnn], dim=1)

        h = []
        
        for lin in self.linear_modules[:-1]:
            x = lin(x)
            h.append(x)
            x = self.layer_norm(x)
            x = self.relu(x)
            x = self.dropout(x)
        
        x = self.relu(h[-1] + h[-2])
        x = self.linear_modules[-1](x)
        return x


def train(model, data_loader, device, optimizer, criterion, n_epoch):

    logger.info('Epoch #{}\n'.format(n_epoch+1))

    train_losses = []
    train_labels = []
    train_predictions = []

    progress_bar = tqdm(total=math.ceil(len(data_loader.dataset)/data_loader.batch_size), desc='Epoch {}'.format(n_epoch + 1))

    model.train()

    for _, data in enumerate(data_loader, 0):


        input_ids = data["source_ids"].to(device)
        attention_mask = data["source_mask"].to(device)
        labels = data['labels'].to(device)

        optimizer.zero_grad()

        pred = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(pred, labels)
              
        loss.backward()      
        optimizer.step()
        
        lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
        scheduler.step()
        
        predict = torch.log_softmax(pred, dim=1).argmax(dim=1)
        
        train_losses.append(loss.item())
        train_labels.extend(labels.cpu().detach().numpy())
        train_predictions.extend(predict.cpu().detach().numpy())

        progress_bar.set_postfix(loss=np.mean(train_losses))
        progress_bar.update(1)
    
    progress_bar.update(1)
    progress_bar.close()
    
    print('\n\nMean Loss after epoch #{0} - {1}'.format(str(n_epoch + 1), np.mean(train_losses)))
    print('F1 score after epoch #{0} on train - {1}\n'.format(str(n_epoch + 1), f1_score(train_labels, train_predictions, average='macro')))
    print('Accuracy score after epoch #{0} on train - {1}\n'.format(str(n_epoch + 1), accuracy_score(train_labels, train_predictions)))

    print(classification_report(train_labels, train_predictions))
    return train_labels, train_predictions


def validating(model, data_loader, device, criterion, n_epoch):

    val_losses, val_labels, val_predictions = [], [], []

    progress_bar = tqdm(total=math.ceil(len(data_loader.dataset)/data_loader.batch_size),
                        desc='Epoch {}'.format(n_epoch + 1))

    model.eval()

    for _, data in enumerate(data_loader, 0):
          
        input_ids = data["source_ids"].to(device)
        attention_mask = data["source_mask"].to(device)
        labels = data['labels'].to(device)

        with torch.no_grad():
            pred = model(input_ids, attention_mask)

            loss = criterion(pred, labels)
          
            predict = torch.log_softmax(pred, dim=1).argmax(dim=1)

        val_losses.append(loss.item())
        val_labels.extend(labels.cpu().detach().numpy())
        val_predictions.extend(predict.cpu().detach().numpy())

        progress_bar.set_postfix(loss=np.mean(val_losses))
        progress_bar.update(1)

    progress_bar.update(1)
    progress_bar.close()
    
    
    valid_stats.append(
        {
            'Val Loss': np.mean(val_losses)
        }
    )

    print('\n\nMean Loss after epoch #{0} - {1}'.format(str(n_epoch + 1), np.mean(val_losses)))
    print('F1 score after epoch #{0} on validation - {1}\n'.format(str(n_epoch + 1), f1_score(val_labels, val_predictions, average='macro')))
    print('Accuracy score after epoch #{0} on validation - {1}\n'.format(str(n_epoch + 1), accuracy_score(val_labels, val_predictions)))
    
    print(classification_report(val_labels, val_predictions))
    return valid_stats


def test(data_loader, device, id_name):
    
    model.load_state_dict(torch.load(f'results/model_with_cnn_justcnn_{id_name}.pth'))
    
    test_labels, test_predictions = [], []

    model.eval()

    for _, data in enumerate(data_loader, 0):
          
        input_ids = data["source_ids"].to(device)
        attention_mask = data["source_mask"].to(device)
        labels = data['labels'].to(device)

        with torch.no_grad():
            pred = model(input_ids, attention_mask)
          
        predict = torch.log_softmax(pred, dim=1).argmax(dim=1)

        test_labels.extend(labels.cpu().detach().numpy())
        test_predictions.extend(predict.cpu().detach().numpy())

    print('F1 macro score on test - {0}\n'.format(f1_score(test_labels, test_predictions, average='macro')))
    print('F1 score on test - {0}\n'.format(f1_score(test_labels, test_predictions, average='weighted')))
    print('Accuracy score on test - {0}\n'.format(accuracy_score(test_labels, test_predictions)))
    
    print(classification_report(test_labels, test_predictions))


def evaluate(model, train_dataset, val_dataset, device, epochs, target_value, weights):
    
    model = model.to(device)
    lr = 2e-4 
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(weight=weights,reduction='mean').to(device)
    global lr_list
    lr_list = []
    global scheduler
    total_steps = len(train_dataset) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=int(total_steps * 0.1),
                                                num_training_steps=total_steps)
    global valid_stats
    valid_stats = []
    best_valid_loss = float('inf')

    for epoch in range(epochs):
        # train
        try:
            train(model, train_dataset, device, optimizer, criterion,  epoch)
            # # validate
            validating(model, val_dataset, device, criterion, epoch)

            if valid_stats[epoch]['Val Loss'] < best_valid_loss:
                best_valid_loss = valid_stats[epoch]['Val Loss']

                name_to_save = f'model_with_cnn_nolayernorm_{target_value}'
                if os.path.isfile('results/'+name_to_save+'.pth'):
                    os.remove('results/'+name_to_save+'.pth')
                    torch.save(model.state_dict(), 'results/'+name_to_save+'.pth')
                else:
                    if not os.path.isdir('results'):
                        os.mkdir('results')
                    torch.save(model.state_dict(), 'results/'+name_to_save+'.pth')
        except KeyboardInterrupt:
            break

In [17]:
%cd  /content/drive/MyDrive/вышка/диплом

/content/drive/MyDrive/вышка/диплом


In [24]:
path_to_data = 'dataset_all_nlp_features_target_classes_no_naive (1).csv'
target_value = 'verb'
path_to_model = 'DeepPavlov/rubert-base-cased'
epochs = 20
seed_value = 42

_ = seed_everything(seed_value)
LR = 2e-5
maxlength = 120
minlength = 2
bsize = 8

dataset = pd.read_csv(path_to_data, sep='\t')

dataset = dataset[dataset.question_id != '129_Чтение текста - видео']

if target_value == 'raven':
            dataset = dataset[dataset["raven"] > 0]
if target_value == 'verb':
            dataset = dataset[dataset["verb"] > 0]
        
tokenizer = AutoTokenizer.from_pretrained(path_to_model)

intelligence = target_value+'_classes'

dataset = dataset[dataset['N_words'] > minlength]

dataset[intelligence] = dataset[intelligence].astype(int)

train_data, extra_data = train_test_split(dataset, test_size=0.22,
                                        stratify=dataset[intelligence],
                                        random_state=seed_value)

vaild_data, test_data = train_test_split(extra_data, test_size=0.45,
                                        stratify=extra_data[intelligence],
                                        random_state=seed_value)


train_dataset_data = DataPreparation(
            tokenizer=tokenizer,
            data = train_data,
            intelligence = intelligence,
            max_length = maxlength
        )

val_dataset_data = DataPreparation(
            tokenizer=tokenizer,
            data = vaild_data,
            intelligence = intelligence,
            max_length = maxlength
        )

test_dataset_data = DataPreparation(
            tokenizer=tokenizer,
            data = test_data,
            intelligence = intelligence,
            max_length = maxlength
        )

weights = class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(dataset[intelligence]), y=dataset[intelligence].to_numpy())

wights_tensor = torch.tensor(weights,dtype=torch.float)

train_dataset = DataLoader(train_dataset_data, batch_size=bsize, drop_last=True, shuffle=True)
val_dataset = DataLoader(val_dataset_data, batch_size=bsize)
test_dataset = DataLoader(test_dataset_data, batch_size=bsize)
        
model = BIGRU_BILSTM_CNN(pre_trained=path_to_model, out_features=len(dataset[intelligence].unique()))


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
evaluate(model=model, train_dataset=train_dataset, val_dataset=val_dataset, device=device, epochs=epochs, target_value=target_value, weights=wights_tensor)

<ipython-input-24-3f032fecbeff>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[intelligence] = dataset[intelligence].astype(int)
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassifi

Epoch 1:   0%|          | 0/632 [00:00<?, ?it/s]



Mean Loss after epoch #1 - 1.6078167687127405
F1 score after epoch #1 on train - 0.19500166531004215

Accuracy score after epoch #1 on train - 0.2014659270998415

              precision    recall  f1-score   support

           0       0.24      0.32      0.27       946
           1       0.15      0.29      0.20       771
           2       0.27      0.12      0.16      1282
           3       0.25      0.07      0.11      1258
           4       0.18      0.33      0.24       791

    accuracy                           0.20      5048
   macro avg       0.22      0.22      0.20      5048
weighted avg       0.23      0.20      0.19      5048



Epoch 1:   0%|          | 0/98 [00:00<?, ?it/s]



Mean Loss after epoch #1 - 1.5610994806095047
F1 score after epoch #1 on validation - 0.21272485769993293

Accuracy score after epoch #1 on validation - 0.2413793103448276

              precision    recall  f1-score   support

           0       0.31      0.41      0.35       147
           1       0.18      0.38      0.24       120
           2       0.22      0.01      0.02       198
           3       0.28      0.07      0.11       195
           4       0.25      0.55      0.34       123

    accuracy                           0.24       783
   macro avg       0.25      0.28      0.21       783
weighted avg       0.25      0.24      0.19       783



...



Mean Loss after epoch #19 - 1.57714920141259
F1 score after epoch #19 on validation - 0.3071399528156998

Accuracy score after epoch #19 on validation - 0.3090676883780332

              precision    recall  f1-score   support

           0       0.37      0.45      0.41       147
           1       0.19      0.25      0.22       120
           2       0.34      0.28      0.31       198
           3       0.32      0.25      0.28       195
           4       0.31      0.33      0.32       123

    accuracy                           0.31       783
   macro avg       0.31      0.31      0.31       783
weighted avg       0.31      0.31      0.31       783



Epoch 20:   0%|          | 0/632 [00:00<?, ?it/s]



Mean Loss after epoch #20 - 1.2651397160796092
F1 score after epoch #20 on train - 0.45024384365382203

Accuracy score after epoch #20 on train - 0.45106973058637084

              precision    recall  f1-score   support

           0       0.49      0.64      0.56       946
           1       0.37      0.45      0.40       770
           2       0.48      0.35      0.40      1282
           3       0.48      0.36      0.41      1259
           4       0.43      0.53      0.47       791

    accuracy                           0.45      5048
   macro avg       0.45      0.47      0.45      5048
weighted avg       0.46      0.45      0.45      5048



Epoch 20:   0%|          | 0/98 [00:00<?, ?it/s]



Mean Loss after epoch #20 - 1.5703312414033073
F1 score after epoch #20 on validation - 0.30717084635582265

Accuracy score after epoch #20 on validation - 0.3116219667943806

              precision    recall  f1-score   support

           0       0.37      0.47      0.41       147
           1       0.19      0.20      0.20       120
           2       0.36      0.30      0.33       198
           3       0.28      0.23      0.25       195
           4       0.32      0.37      0.34       123

    accuracy                           0.31       783
   macro avg       0.30      0.32      0.31       783
weighted avg       0.31      0.31      0.31       783



In [25]:
def test(data_loader, device, id_name):
    # model = model.to(device)
#     model = BertBaseline_ResNet(model_name=path_to_model, out_features=len(dataset[intelligence].unique()))
    
    # model = BIGRU_BILSTM_CNN(pre_trained=path_to_model, out_features=len(dataset[intelligence].unique()))

    model.load_state_dict(torch.load(f'results/model_with_cnn_nolayernorm_{id_name}.pth'))
    
    # model = model.to(device)
    test_labels, test_predictions = [], []

    model.eval()

    for _, data in enumerate(data_loader, 0):
          
        input_ids = data["source_ids"].to(device)
        attention_mask = data["source_mask"].to(device)
        labels = data['labels'].to(device)

        with torch.no_grad():
            pred = model(input_ids, attention_mask)
          
        predict = torch.log_softmax(pred, dim=1).argmax(dim=1)

        test_labels.extend(labels.cpu().detach().numpy())
        test_predictions.extend(predict.cpu().detach().numpy())

    print('F1 macro score on test - {0}\n'.format(f1_score(test_labels, test_predictions, average='macro')))
    print('F1 score on test - {0}\n'.format(f1_score(test_labels, test_predictions, average='weighted')))
    print('Accuracy score on test - {0}\n'.format(accuracy_score(test_labels, test_predictions)))
    
    print(classification_report(test_labels, test_predictions))

In [26]:
device

device(type='cuda')

In [30]:
test(train_dataset, device, target_value) 

F1 macro score on test - 0.3171624387302704

F1 score on test - 0.3297144777208272

Accuracy score on test - 0.35439778129952454

              precision    recall  f1-score   support

           0       0.37      0.62      0.46       945
           1       0.25      0.04      0.08       770
           2       0.34      0.35      0.35      1282
           3       0.36      0.37      0.36      1259
           4       0.36      0.32      0.34       792

    accuracy                           0.35      5048
   macro avg       0.34      0.34      0.32      5048
weighted avg       0.34      0.35      0.33      5048



In [ ]:
test(val_dataset, device, target_value) 

In [32]:
test(val_dataset, device, target_value) 

F1 macro score on test - 0.30953606944260775

F1 score on test - 0.31903002309898026

Accuracy score on test - 0.3371647509578544

              precision    recall  f1-score   support

           0       0.35      0.56      0.43       147
           1       0.43      0.08      0.14       120
           2       0.34      0.37      0.35       198
           3       0.32      0.32      0.32       195
           4       0.32      0.28      0.30       123

    accuracy                           0.34       783
   macro avg       0.35      0.32      0.31       783
weighted avg       0.35      0.34      0.32       783



In [31]:
test(test_dataset, device, target_value) 

F1 macro score on test - 0.29015688430170494

F1 score on test - 0.30260773571799193

Accuracy score on test - 0.32710280373831774

              precision    recall  f1-score   support

           0       0.37      0.59      0.46       120
           1       0.31      0.04      0.07        98
           2       0.30      0.33      0.31       163
           3       0.32      0.35      0.34       160
           4       0.29      0.26      0.27       101

    accuracy                           0.33       642
   macro avg       0.32      0.31      0.29       642
weighted avg       0.32      0.33      0.30       642

